In [5]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch climate data
def get_climate_data(lat, lon, start_date, end_date):
    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    
    all_records = []  # Store data for all years

    # Fetch data in yearly chunks to avoid API overload
    current_start = start_date
    while current_start <= end_date:
        current_end = min(current_start.replace(year=current_start.year + 1) - timedelta(days=1), end_date)
        
        params = {
            "parameters": "T2M,T2M_MIN,T2M_MAX,PRECTOTCORR,RH2M",
            "community": "RE",
            "longitude": lon,
            "latitude": lat,
            "start": current_start.strftime("%Y%m%d"),
            "end": current_end.strftime("%Y%m%d"),
            "format": "JSON"
        }

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
            data = response.json()
            
            # Validate data structure
            if "properties" not in data or "parameter" not in data["properties"]:
                print(f"Unexpected response format for {current_start.year}. Skipping...")
                current_start = current_start.replace(year=current_start.year + 1)
                continue

            parameters = data["properties"]["parameter"]
            
            # Extract records
            for date, temp_mean in parameters.get("T2M", {}).items():
                all_records.append({
                    "date": datetime.strptime(date, "%Y%m%d"),
                    "temp_mean": temp_mean,
                    "temp_min": parameters.get("T2M_MIN", {}).get(date, None),
                    "temp_max": parameters.get("T2M_MAX", {}).get(date, None),
                    "rainfall": parameters.get("PRECTOTCORR", {}).get(date, None),
                    "humidity": parameters.get("RH2M", {}).get(date, None)
                })

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {current_start.year}: {e}")
        
        # Move to the next year
        current_start = current_start.replace(year=current_start.year + 1)

    return pd.DataFrame(all_records)

# Define location and date range
latitude, longitude = 8.4281, 33.7833  # Lankien, South Sudan
start_date = datetime(2015, 1, 1)
end_date = datetime(2024, 12, 31)

# Fetch climate data
df_climate = get_climate_data(latitude, longitude, start_date, end_date)

# Create output directory
output_folder = "Temp-rain-humid"
os.makedirs(output_folder, exist_ok=True)

# Save data to CSV
output_path = os.path.join(output_folder, "NASA-Temp-perc-humid.csv")
df_climate.to_csv(output_path, index=False)

print(f"Climate data saved to {output_path}")


Climate data saved to Temp-rain-humid/NASA-Temp-perc-humid.csv
